In [32]:
import tensorflow as tf
import numpy as np
import helper

In [191]:
def batch(inputs, max_sequence_length=None):
    sequence_lengths = [len(seq) for seq in inputs]
    batch_size = len(inputs)

    if max_sequence_length is None:
        max_sequence_length = max(sequence_lengths)

    inputs_batch_major = np.zeros(shape=[batch_size, max_sequence_length], dtype=np.float32)

    for i, seq in enumerate(inputs):
        for j, element in enumerate(seq):
            inputs_batch_major[i,j] = element

    inputs_time_major = inputs_batch_major.swapaxes(0,1)

    return inputs_time_major, sequence_lengths

def random_sequences(length_from, length_to, vocab_lower, vocab_upper, batch_size):
    if length_from > length_to:
        raise ValueError('length_from > length_to')

    def random_length():
        if length_from == length_to:
            return length_from
        return np.random.randint(length_from, length_to + 1)

    while True:
        yield [
        np.random.randint(low = vocab_lower, high=vocab_upper, size=random_length()).tolist()
        for _ in range(batch_size)
        ]

In [207]:
batch_size = 1
batches = random_sequences(length_from=1, length_to=1,
                           vocab_lower=2, vocab_upper=10,
                           batch_size=batch_size)

In [208]:
xs = tf.placeholder(shape = (None, None), dtype = tf.float32)
ys = tf.placeholder(shape = (None, None), dtype = tf.float32)

In [209]:
def next_feed():
    batch = next(batches)
    encoder_inputs_, encoder_input_lengths_ = helper.batch(batch)
    decoder_targets_, _ = helper.batch(
        [sequence for sequence in batch]
    )
    return {
        xs: encoder_inputs_,
        ys: decoder_targets_
    }

In [210]:
def add_layer(inputs, in_size, out_size, activation_function=None):
	Weights = tf.Variable(tf.random_normal([in_size,out_size]))
	biases = tf.Variable(tf.zeros([1,out_size]) + 0.1)
	Wx_plus_b = tf.add(tf.matmul(inputs, Weights), biases)
	if activation_function is None:
		outputs = Wx_plus_b
	else:
		outputs = activation_function(Wx_plus_b)
	return outputs

In [211]:
x_data = np.linspace(-1,1,500)[:,np.newaxis]
noise = np.random.normal(0,0.05,x_data.shape)
y_data = np.square(x_data) - 0.5 + noise

In [212]:
l1 = add_layer(xs, 1, 10, activation_function = tf.nn.relu)
prediction = add_layer(l1, 10, 1, activation_function = None)

In [213]:
loss = tf.reduce_mean(tf.reduce_sum(tf.square(ys - prediction), reduction_indices = [1]))

train_step = tf.train.AdamOptimizer().minimize(loss)

In [214]:
sess = tf.InteractiveSession()

sess.run(tf.global_variables_initializer())

In [215]:
for i in range(3000):
    fd = next_feed()
    sess.run(train_step, fd)
    if i % 100 == 0:
        print(sess.run(loss, fd))

575.57
727.713
23.4731
15.1034
26.1245
219.337
38.671
24.0531
23.0175
0.0847609
10.8155
1.2021
5.43152
0.152596
0.204137
0.0785588
0.608908
1.51474
1.80304
1.457
2.91177
0.792187
0.443708
0.455385
1.48905
0.606025
0.000848917
0.00291197
0.466491
0.173891
